# Face Recognition Model

## Setup

In [12]:
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import shutil
from pathlib import Path
import re
import uuid

In [13]:
# Model(inputs = [inputImg, veriImg], outputs = [1,0] )

In [14]:
gpus = tf.config.experimental.list_physical_devices('GPI')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [15]:
POS = os.path.join('data', 'positive')
NEG = os.path.join('data', 'negative')
ANC = os.path.join('data', 'anchor')

## Preprocess

### Data gathering

In [ ]:
def consolidate_images(source_dir):
    if not os.path.exists(source_dir):
        print(f"Creating directory: {source_dir}")
        os.makedirs(source_dir)

    subfolders = [f.path for f in os.scandir(source_dir) if f.is_dir()]
    
    if not subfolders:
        print(f"No subfolders found in {source_dir}")
        return

    image_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp')
   
    copied_count = 0

    processed_folders = []
   
    for folder in subfolders:
        print(f"Processing folder: {folder}")
        folder_processed = False
       
        for root, dirs, files in os.walk(folder):
            for file in files:
                if file.lower().endswith(image_extensions):
                    source_path = os.path.join(root, file)
                    base_name = os.path.splitext(file)[0]
                    extension = os.path.splitext(file)[1]
                    counter = 1
                    dest_file = file
                   
                    while os.path.exists(os.path.join(source_dir, dest_file)):
                        dest_file = f"{base_name}_{counter}{extension}"
                        counter += 1
                   
                    dest_path = os.path.join(source_dir, dest_file)
                   
                    try:
                        shutil.copy2(source_path, dest_path)
                        copied_count += 1
                        folder_processed = True
                    except Exception as e:
                        print(f"Error copying {file}: {str(e)}")
        
        if folder_processed:
            processed_folders.append(folder)
    
    print(f"\nTotal images copied: {copied_count}")
    
    if processed_folders:
        print("\nDeleting processed folders:")
        for folder in processed_folders:
            try:
                shutil.rmtree(folder)
                print(f"Deleted: {folder}")
            except Exception as e:
                print(f"Error deleting {folder}: {str(e)}")

consolidate_images(NEG)

In [ ]:
# Correcting function because I messed up the top one and did it twice once with folder deleting and once without lol
def remove_duplicate_images(directory):
    if not os.path.exists(directory):
        print(f"Error: Directory '{directory}' does not exist.")
        return
    
    files = os.listdir(directory)
    files_to_delete = []
    pattern = re.compile(r'^(.+)_1(\.[^.]+)$')
    deleted_count = 0
    
    print(f"Scanning directory: {directory}")

    for file in files:
        match = pattern.match(file)
        if match:
            base_name = match.group(1)
            extension = match.group(2)
            original_file = f"{base_name}{extension}"
            if original_file in files:
                files_to_delete.append(file)

    for file in files_to_delete:
        try:
            file_path = os.path.join(directory, file)
            os.remove(file_path)
            print(f"Deleted: {file}")
            deleted_count += 1
        except Exception as e:
            print(f"Error deleting {file}: {str(e)}")
    
    print(f"\nTotal duplicate files deleted: {deleted_count}")

remove_duplicate_images(NEG)

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        print("Failed to grab frame")
        break

    frame = frame[120:120+250, 200:200+250, :]
    cv2.imshow('Image Collection', frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('a'):
        img_name = os.path.join(ANC, f'{uuid.uuid1()}.jpg')
        cv2.imwrite(img_name, frame)
        print(f"Saved anchor image: {img_name}")
    elif key == ord('p'):
        img_name = os.path.join(POS, f'{uuid.uuid1()}.jpg')
        cv2.imwrite(img_name, frame)
        print(f"Saved positive image: {img_name}")
    elif key == ord('q'):
        print("Quitting...")
        break
    
cap.release()
cv2.destroyAllWindows()

### Preprocess

In [68]:
anchor = tf.data.Dataset.list_files(ANC+'\*.jpg').take(300)
positive = tf.data.Dataset.list_files(POS+'\*.jpg').take(300)
negative = tf.data.Dataset.list_files(NEG+'\*.jpg').take(300)

<>:1: SyntaxWarning: invalid escape sequence '\*'
<>:2: SyntaxWarning: invalid escape sequence '\*'
<>:3: SyntaxWarning: invalid escape sequence '\*'
<>:1: SyntaxWarning: invalid escape sequence '\*'
<>:2: SyntaxWarning: invalid escape sequence '\*'
<>:3: SyntaxWarning: invalid escape sequence '\*'
C:\Users\lokna\AppData\Local\Temp\ipykernel_31980\2097756147.py:1: SyntaxWarning: invalid escape sequence '\*'
  anchor = tf.data.Dataset.list_files(ANC+'\*.jpg').take(300)
C:\Users\lokna\AppData\Local\Temp\ipykernel_31980\2097756147.py:2: SyntaxWarning: invalid escape sequence '\*'
  positive = tf.data.Dataset.list_files(POS+'\*.jpg').take(300)
C:\Users\lokna\AppData\Local\Temp\ipykernel_31980\2097756147.py:3: SyntaxWarning: invalid escape sequence '\*'
  negative = tf.data.Dataset.list_files(NEG+'\*.jpg').take(300)


In [69]:
dir_test = anchor.as_numpy_iterator()

In [70]:
def preprocess(img_path):
    byte_img = tf.io.read_file(img_path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img,(100,100))
    img = img/255
    return img

In [71]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

In [72]:
def preproc_twin(in_img, valid_img, label):
    return(preprocess(in_img), preprocess(valid_img), label)

In [73]:
data = data.map(preproc_twin)
data = data.cache()
data = data.shuffle(buffer_size=1024)

In [74]:
train_data = data.take(round(len(data)*0.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [75]:
test_data = data.skip(round(len(data)*0.7))
test_data = test_data.take(round(len(data)*0.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

## Model

### Building

In [ ]:
def embeding_make():
    in_ = Input(shape=(100,100,3), name="in img")

    c1 = Conv2D(64, (10,10), activation='relu')(in_)
    p1 = MaxPooling2D(64, (2,2), padding='same')(c1)

    c2 = Conv2D(128, (7,7), activation='relu')(p1)
    p2 = MaxPooling2D(64, (2,2), padding='same')(c2)
    
    c3 = Conv2D(128, (4,4), activation='relu')(p2)
    p4 = MaxPooling2D(64, (2,2), padding='same')(c3)

    c4 = Conv2D(256, (4,4), activation='relu')(p3)
    f1 = Flatten()(c4)
    d1 = Dense(4096,activation='sigmoid')(f1)

    return Model(input=[in_],output=[d1],name=['embedding'])

### Training